Aluno: Giovanni Bruno Travassos de Carvalho - 11506849

Resolução da primeira questão da primeira lista de exercícios de Deep Learning

Professor: Tiago Maritan 

## Questão 1 
A representação de uma determinada mensagem digital ternária, isto é formada por três bits,
forma um cubo cujos vértices correspondem a mesma representação digital. Supondo que ao
transmitirmos esta mensagem a mesma possa ser contaminada por ruído formado em torno de
cada vértice uma nuvem esférica de valores aleatórios com raio máximo é 0.1. Formule este
problema como um problema de classificação de padrões e treine uma rede Perceptron de
Rosenblatt (Perceptron de camada única) para atuar como classificador/decodificador. Para
solução do problema defina antes um conjunto de treinamento e um conjunto de validação.

Dica: O problema pode ser formulado como um problema de classificação de 8 padrões diferentes, sendo que cada padrão representa um vértice do cubo. 

Padrão 1: x = {0,0,0} com vetor resposta d = {1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0}

Padrão 2: x = {0,0,1} com vetor resposta d = {-1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0}

Padrão 3: x = {0,1,0} com vetor resposta d = {-1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0}

Padrão 4: x = {0,1,1} com vetor resposta d = {-1.0, -1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0}

Padrão 5: x = {1,0,0} com vetor resposta d = {-1.0, -1.0, -1.0, -1.0, 1.0, -1.0, -1.0, -1.0}

Padrão 6: x = {1,0,1} com vetor resposta d = {-1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, -1.0}

Padrão 7: x = {1,1,0} com vetor resposta d = {-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0}

Padrão 8: x = {1,1,1} com vetor resposta d = {-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0}

Dica: -1 pode ser trocado para 0, mudando a função de ativação 

### Primeira Etapa: Criação do Arquivo de Entrada 

Objetivo: Criar um conjunto de dados seguindo os padrões, porém com variação randômica de ruído em algumas instâncias. 
    Serão criados 800 entradas. 
    

In [5]:
import numpy as np 
import random 

random.seed(234) 
    # Criada uma semente para garantir que sempre seja obtido o mesmo conjunto de dados 
    
padroes = [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], 
           [1, 0, 0], [1, 0, 1], [1, 1, 0], [1, 1, 1]] # 8 tipos de padrões normais

ruidoMax = 0.1 
    # Ruído máximo de 0.1 

amostras = 100 
    # Número de amostras para cada tipo de entrada 
    
count=0
coluna = len(padroes[0])
linha = len(padroes)*amostras
ruido = [0]*coluna 

arquivo = open('./entradas.txt', 'w')
lista = []

for i in range (0, len(padroes)): 
    
    for j in range(0,amostras): 
        
        for k in range(0,coluna):
            ruido[k] = random.randrange(0,2) 
                # 0 = sem ruído
                # 1 = com ruído
        for k in range(0,coluna): 
            if(ruido[k]==0):
                a = padroes[i][k]
                arquivo.write(str(a))
                lista.append(a)
            else:
                a = padroes[i][k] + random.uniform(-ruidoMax, ruidoMax)
                arquivo.write(str(a)) # Acrescenta o ruído 
                lista.append(a)
            if(k==2):
                arquivo.write('\n')
            else: 
                arquivo.write(',')
        count+=1 

arquivo.close()

### Segunda Etapa: Criação do Arquivo de Saída

Objetivo: Para cada linha do arquivo de entrada, encontrar o seu binário de 8bits equivalente. 


In [2]:
tamSaida = len(lista)
i = 0; aux = 0
codeBin = [0]*coluna 
peso = [2,1,0]
saida = [0]*8 

arquivo2 = open('./saida.txt','w')

while(i < tamSaida):
    
    for j in range (0,coluna): 
        if(lista[i] <= 0.1): 
            # Valores que forem menores que 0.1 são equivalentes a 0 
            codeBin[j]=0
        else: 
            # Valores maiores que 0.9 são equivalentes a 1 
            codeBin[j]=1
        i+=1
    
    for g in range(coluna-1, -1, -1):
        aux += codeBin[g] * (2 **peso[g])
        # Calcula o número decimal equivalente ao binário da saída 
    
    saida[aux] = 1 # Seta o bit para 1 
    
    for k in range (0,8):
        arquivo2.write(str(saida[k]))
        
        if(k!=7):
            arquivo2.write(',')
        else: 
            arquivo2.write("\n")
            
    aux = 0
    saida=[0]*8 # Seta para -1 o vetor de saída para ser reescrito 

arquivo2.close()

Nas primeiras etapas foram criados dois arquivos com 800 instâncias cada, sendo 100 para cada tipo de padrão. 

### Terceira Etapa: Rede Neural - Perceptron de Rosenblatt 

In [9]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
    # Importação das bibliotecas 

In [6]:
X = []
X = np.loadtxt('../entradas.txt', dtype=float, delimiter=',', encoding = 'bytes')

Y = []
Y = np.loadtxt('../saida.txt', dtype=float, delimiter=',', encoding = 'bytes')

In [7]:
print(X)

[[ 0.0715832   0.08466661  0.        ]
 [-0.0306616   0.         -0.00156604]
 [ 0.         -0.02204448  0.03691852]
 ...
 [ 1.          0.95714442  1.        ]
 [ 1.0076904   0.96645834  1.        ]
 [ 1.08147671  0.95129295  0.992824  ]]


In [8]:
print(Y)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [10]:
# Dividindo o conjunto em dados de Treino e de Teste: 

X_Treino, X_Teste, Y_Treino, Y_Teste = train_test_split(X, Y, test_size=0.2, random_state=0)

In [11]:
# Normalização com Standard Scaler 
# Standard Scaler implementa a TransformerAPI para calcular a média e o desvio padrão em um conjunto de treinamento

ss = StandardScaler()
X_Treino = ss.fit_transform(X_Treino)
X_Teste = ss.fit_transform(X_Teste)

In [ ]:
print(X_Teste)

In [13]:
# Criando a Rede Neural: 

redeNeural = tf.keras.models.Sequential()
    # Como é um Perceptron Simples, só tem a camada de entrada e a saída 
redeNeural.add(tf.keras.layers.Dense(units=3, activation='relu')) # Camada de Entrada
redeNeural.add(tf.keras.layers.Dense(units=8, activation='sigmoid')) # Camada de Saída 

In [14]:
# Compilando a Rede Neural: 

redeNeural.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [16]:
# Treinando os dados de Teste da Rede Neural: 

treino = redeNeural.fit(X_Treino, Y_Treino, validation_split=0.2, epochs=100, batch_size=10)

Train on 512 samples, validate on 128 samples
Epoch 1/100
512/512 [==============================] - 0s 219us/sample - loss: 0.0927 - acc: 0.9780 - val_loss: 0.0978 - val_acc: 0.9717
Epoch 2/100
512/512 [==============================] - 0s 383us/sample - loss: 0.0914 - acc: 0.9778 - val_loss: 0.0966 - val_acc: 0.9717
Epoch 3/100
512/512 [==============================] - 0s 287us/sample - loss: 0.0902 - acc: 0.9783 - val_loss: 0.0953 - val_acc: 0.9727
Epoch 4/100
512/512 [==============================] - 0s 285us/sample - loss: 0.0890 - acc: 0.9783 - val_loss: 0.0940 - val_acc: 0.9727
Epoch 5/100
512/512 [==============================] - 0s 320us/sample - loss: 0.0878 - acc: 0.9790 - val_loss: 0.0929 - val_acc: 0.9727
Epoch 6/100
512/512 [==============================] - 0s 264us/sample - loss: 0.0866 - acc: 0.9790 - val_loss: 0.0917 - val_acc: 0.9736
Epoch 7/100
512/512 [==============================] - 0s 219us/sample - loss: 0.0855 - acc: 0.9790 - val_loss: 0.0905 - val_acc: 0.

A Rede Neural foi treinada com 100 épocas, e teve acurácia de 97% para os dados de Treino.

In [17]:
# Previsão com os dados de Teste: 

previsao = redeNeural.predict(X_Teste) 

In [19]:
# Matriz de confusão para ver os acertos e erros: 

confusao = confusion_matrix(Y_Teste.argmax(axis=1), previsao.argmax(axis=1))
print(confusao)

[[20  0  0  0  0  0  0  0]
 [ 0 12  0  0  0  0  0  0]
 [ 0  0 26  0  0  0  0  0]
 [ 0  0  0 14  0  0  0  0]
 [ 0  0  0  0 28  0  0  0]
 [ 0  0  0  0  0 15  0  0]
 [ 0  0  0  0  0  0 20  0]
 [ 0  0  0  0  0  0  0 25]]


In [ ]:
# Calculando a Acurácia: 

accuracy_score(Y_Teste, previsao)